#Actividad 2: KDD

##Forest y Clasificación

In [1]:
import pandas as pd 
import numpy as np
from sklearn import tree
import plotly.express as px

Dataset de Cancer de Mama sacado de Kaggle. Los datos ya están curados. 

Importa ver las características de las pacientes y relacionarlas con el Status. 

In [2]:
path='/content/drive/MyDrive/1 - Carpeta/4 - IA/Colab de IA/Pruebas - Propias/Breast_Cancer.csv'
df=pd.read_csv(path)
df.head()

,Age,Race,Marital Status,T Stage,N Stage,6th Stage,differentiate,Grade,A Stage,Tumor Size,Estrogen Status,Progesterone Status,Regional Node Examined,Reginol Node Positive,Survival Months,Status
0,68,White,Married,T1,N1,IIA,Poorly differentiated,3,Regional,4,Positive,Positive,24,1,60,Alive
1,50,White,Married,T2,N2,IIIA,Moderately differentiated,2,Regional,35,Positive,Positive,14,5,62,Alive
2,58,White,Divorced,T3,N3,IIIC,Moderately differentiated,2,Regional,63,Positive,Positive,14,7,75,Alive
3,58,White,Married,T1,N1,IIA,Poorly differentiated,3,Regional,18,Positive,Positive,2,1,84,Alive
4,47,White,Married,T2,N1,IIB,Poorly differentiated,3,Regional,41,Positive,Positive,3,1,50,Alive


In [3]:
def fig (df,var):
    fig2 = px.histogram(df,x=var,color=var,template='plotly_dark')
    fig2.show()

In [4]:
for i in df.columns:
    fig (df,i)

In [5]:
for col in df.columns:
    print("-",col, df[col].unique(),'\n')

- Age [68 50 58 47 51 40 69 46 65 48 62 61 56 43 60 57 55 63 66 53 59 54 49 64
 42 37 67 31 52 33 45 38 39 36 41 44 32 34 35 30] 

- Race ['White' 'Black' 'Other'] 

- Marital Status ['Married' 'Divorced' 'Single ' 'Widowed' 'Separated'] 

- T Stage  ['T1' 'T2' 'T3' 'T4'] 

- N Stage ['N1' 'N2' 'N3'] 

- 6th Stage ['IIA' 'IIIA' 'IIIC' 'IIB' 'IIIB'] 

- differentiate ['Poorly differentiated' 'Moderately differentiated' 'Well differentiated'
 'Undifferentiated'] 

- Grade ['3' '2' '1' ' anaplastic; Grade IV'] 

- A Stage ['Regional' 'Distant'] 

- Tumor Size [  4  35  63  18  41  20   8  30 103  32  13  59  15  19  46  24  25  29
  40  70  22  50  17  21  10  27  23   5  51   9  55 120  77   2  11  12
  26  75 130  34  80   3  60  14  16  45  36  76  38  49   7  72 100  43
  62  37  68  52  85  57  39  28  48 110  65   6 105 140  42  31  90 108
  98  47  54  61  74  33   1  87  81  58 117  44 123 133  95 107  92  69
  56  82  66  78  97  88  53  83 101  84 115  73 125 104  94  86  64  96

In [6]:
df.columns

Index(['Age', 'Race', 'Marital Status', 'T Stage ', 'N Stage', '6th Stage',
       'differentiate', 'Grade', 'A Stage', 'Tumor Size', 'Estrogen Status',
       'Progesterone Status', 'Regional Node Examined',
       'Reginol Node Positive', 'Survival Months', 'Status'],
      dtype='object')

In [7]:
#Elimino información que pienso que es irrelevante como Estado marital y clasificación de AMC
df.drop(columns=['Marital Status'],inplace=True)
df.drop(columns=['6th Stage'],inplace=True)

Paso a valores continuos todos mis categoricos.

In [8]:
#reemplazo de variables categoricas a un valor continuo según su predominio o importancia 
df['T Stage ']=df['T Stage '].replace({'T1':1,'T2':2,'T3':3,'T4':4})
df['N Stage']=df['N Stage'].replace({'N1':1,'N2':2,'N3':3})
df['Grade']=df['Grade'].replace({'3':3,'2':2,'1':1,' anaplastic; Grade IV':4})
df['Estrogen Status']=df['Estrogen Status'].replace({'Positive':1,'Negative':0})
df['Progesterone Status']=df['Progesterone Status'].replace({'Positive':1,'Negative':0})
df['A Stage']=df['A Stage'].replace({'Regional':0,'Distant':1})
df['Race']=df['Race'].replace({'White':0,'Black':1,'Other':2})
df['Status']=df['Status'].replace({'Dead':0,'Alive':1})
df['differentiate']=df['differentiate'].replace({'Undifferentiated':0,'Poorly differentiated':1, 'Moderately differentiated':2,'Well differentiated':3})

df.head()

,Age,Race,T Stage,N Stage,differentiate,Grade,A Stage,Tumor Size,Estrogen Status,Progesterone Status,Regional Node Examined,Reginol Node Positive,Survival Months,Status
0,68,0,1,1,1,3,0,4,1,1,24,1,60,1
1,50,0,2,2,2,2,0,35,1,1,14,5,62,1
2,58,0,3,3,2,2,0,63,1,1,14,7,75,1
3,58,0,1,1,1,3,0,18,1,1,2,1,84,1
4,47,0,2,1,1,3,0,41,1,1,3,1,50,1


Defino mi target el cual es el Status. Quiero establecer una relación entre los valores de entrada (características del tumor de la mujer) y el Status de Viva o Fallecida. 

Información importante para el caso de prevención (como problema).


In [9]:
target=df['Status']
X=df.drop(['Status'],axis=1,inplace=False)

In [10]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,target,test_size=0.3,random_state=42)
arbol = tree.DecisionTreeClassifier()
arbol

DecisionTreeClassifier()

In [11]:
arbol.fit(X_train,y_train)

DecisionTreeClassifier()

In [12]:
arbol.score(X_test, y_test)

0.8261589403973509

In [13]:
import graphviz 
dot_data = tree.export_graphviz(arbol, out_file=None) 
graph = graphviz.Source(dot_data) 
graph.render("arbol") 

'arbol.pdf'

El arbol de decisión en muy complejo como para leer a simple vista. Esto puede ser a la cantidad de variables de entrada y la complejidad de las mismas. 

Presenta a simple vista una varianza alta. 

Dificil de establecer relaciones directas en el ámbito de la oncología.

##Otro ejemplo por stacking: entrenar multiples modelos (XGBoost y Random Forests) para tener un meta modelo más preciso.

In [14]:
from xgboost import XGBClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression

In [15]:
xgb = XGBClassifier(use_label_encoder=False, eval_metric="logloss", random_state=42) #sin etiquetas, metrica de entrenamiento por pérdida logaritmica)
xgb.fit(X_train, y_train) #entrenar el modelo con los mismos datos anteriores (data y target)
xgb_preds_train = xgb.predict_proba(X_train) #toma cada entrada y devuelve la probabilidad de cada fila de que pertenezca a cada clase del target (prob Alive y prob Dead)

/usr/local/lib/python3.10/dist-packages/xgboost/sklearn.py:1395: UserWarning:

`use_label_encoder` is deprecated in 1.7.0.



In [19]:
rf=RandomForestClassifier(n_estimators=100, random_state=42)
rf.fit(X_train, y_train)
rf_preds_train = rf.predict_proba(X_train) #hace lo mismo pero con randomForest

In [20]:
stacked_preds_train = np.hstack((rf_preds_train, xgb_preds_train)) #Concateno horizontalmente ambas matrices de probabilidad
#va a ser el nuevo input del meta modelo
meta_model = LogisticRegression() #pq es más facil 

meta_model.fit(stacked_preds_train, y_train)

LogisticRegression()

In [21]:
#mismo proceso de probabilidad pero con los valores para test
rf_preds_test = rf.predict_proba(X_test)
xgb_preds_test = xgb.predict_proba(X_test)

# Stack las predicciones de los test (mismo que train)
stacked_preds_test = np.hstack((rf_preds_test, xgb_preds_test))

In [22]:
#predicción final del meta modelo (usando como inputs los prob de test)
y_pred = meta_model.predict(stacked_preds_test) 

#Accuracy
accuracy = np.mean(y_pred == y_test) #Proporción de predicciones correctas respecto al total de predicciones realizadas
print(" accuracy:", accuracy)

 accuracy: 0.9081125827814569


Aplicamos el método de Stacking con 2 modelos diferentes para obtener uno nuevo más preciso. La métrica es similar al generado automáticamente con score.arbol (% de aciertos sobre el total).

#Etica

Se debe tener en cuenta en la ética de la IA los posibles riesgos que relacionados a la exposición a las personas a errores y establecer las relaciones causales entre las variables de entrada.


En este caso, el objetivo del algoritmo es académico, buscando comprender cómo funciona el algoritmo, en particular cuando se utiliza XGBoost en un metamodelo. Una de las discusiones relevantes es la aplicación de la IA en el ámbito de la salud, especialmente en lo que respecta a "Salud y Bienestar Social". Esto se debe a que el proceso de toma de decisiones finales no involucra a otros actores, como médicos o familiares. Sin embargo, estas herramientas pueden ser de gran utilidad para impulsar el progreso en el ámbito médico y mejorar la calidad del cuidado de la salud.


El área de **Economía y Mercado Laboral** es un tema importante en el caso de este tipo de algoritmos. Una de las principales preocupaciones del futuro es el reemplazo del trabajo humano por las máquinas. En un principio, se pensaba en el reemplazo del trabajo duro y las "habilidades duras"; sin embargo, con la llegada de los chatbots y los LLM (modelos de lenguaje a gran escala), el enfoque se ha desplazado hacia el reemplazo de las "habilidades blandas". Considerando el caso de Random Forest, es posible imaginar que un algoritmo pueda reemplazar el trabajo de médicos, economistas, consultores, etc., cuyo trabajo consiste en tomar decisiones basadas en datos específicos. Según el texto de la UNESCO, la IA debe contribuir a conformar "procesos educativos acordes a las nuevas demandas del mercado laboral y a fortalecer las habilidades blandas".


El derecho a la privacidad no es menor y se tiene una importancia en las regulaciones europeas (no solamente en el ámbito de la IA). En este caso, donde un algoritmo de IA clasifica de manera supervisada a un grupo de personas, es fundamental el resguardo de esa información. 

Para el uso de los Forest de Sklearn se deben pasar los valores categóricos a continuos. En este caso, se prestó atención en cambiar valores cat. a cont. teniendo en cuenta su importancia. También se tiene en cuenta los dataset muy desbalanceados que implica una mala decisión de los diferentes árboles.

---


